# Datos y librerias

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut

path = "data/ARIA/5.CourseOfTreatment/"
show = 1
cur = ut.leer_csv(path + "_Course.csv", show).drop(columns=["ClinicalProtocolDir","Comment","CompletedByUserName",
                                                            "TransactionId","CourseId","CompletedDateTime"])

pi = ut.leer_csv(path + "_PhysicianIntent.csv", show).drop(columns=["CreationUserName","CreationDate","PhysicianIntentUID"])

# Diagnosis:
cd = ut.leer_csv(path + "CourseDiagnosis.csv", show)
diag = ut.leer_csv(path + "_Diagnosis.csv", show).drop(columns=["DiagnosisTableName","DiagnosisId","Description",
                                                                "HistologyTableName","DateStamp","HisDescription"])
pri_diag = ut.leer_csv(path + "_PrmryDiagnosis.csv", show).drop(columns=["StageCriteria"])
sec_diag = ut.leer_csv(path + "_SecndryDiagnosis.csv", show)

path = "data/ARIA/7.ActivityAndBilling/"
show = 1
a1 = ut.leer_csv(path + "_ActCaptDiagnosis.csv", show)
a2 = ut.leer_csv(path + "_ActivityCapture.csv", show)

Archivo: data/ARIA/5.CourseOfTreatment/_Course.csv
Se dejaron 10 columnas.
Se eliminaron 4 columnas:


Archivo: data/ARIA/5.CourseOfTreatment/_PhysicianIntent.csv
Se dejaron 6 columnas.
Se eliminaron 5 columnas:


Archivo: data/ARIA/5.CourseOfTreatment/CourseDiagnosis.csv
Se dejaron 3 columnas.
Se eliminaron 4 columnas:


Archivo: data/ARIA/5.CourseOfTreatment/_Diagnosis.csv
Se dejaron 12 columnas.
Se eliminaron 5 columnas:


Archivo: data/ARIA/5.CourseOfTreatment/_PrmryDiagnosis.csv
Se dejaron 7 columnas.
Se eliminaron 2 columnas:


Archivo: data/ARIA/5.CourseOfTreatment/_SecndryDiagnosis.csv
Se dejaron 1 columnas.
Se eliminaron 1 columnas:


Archivo: data/ARIA/7.ActivityAndBilling/_ActCaptDiagnosis.csv
Se dejaron 8 columnas.
Se eliminaron 7 columnas:


Archivo: data/ARIA/7.ActivityAndBilling/_ActivityCapture.csv
Se dejaron 10 columnas.
Se eliminaron 12 columnas:




# Exploración

In [19]:
# ut.resumen_tabla(cur)
# ut.info_tabla(cur)

# Ordeno diagnosis primarias y secundarias

In [20]:
diag_pri = diag[diag.DiagnosisType=="PrmryDiagnosis"]\
.merge(pri_diag, on="DiagnosisSer", how="outer", suffixes = ("_diag", "_pri_diag")).drop(columns="DiagnosisType")

diag_sec = diag[diag.DiagnosisType=="SecndryDiagnosis"]\
.merge(sec_diag, on="DiagnosisSer", how="outer", suffixes = ("_diag", "_sec_diag")).drop(columns="DiagnosisType")

diag_sec = diag_sec.rename(columns={'DiagnosisCode': 'DiagnosisCode_sec',
                                    'ObjectStatus': 'ObjectStatus_sec'}).drop(columns="HistologyCode")


diag_pri = diag_pri.drop(columns="PatientSer")
diag_sec = diag_sec.drop(columns="PatientSer")

# Relacción Course-Diagnosis
DiagnosisSer puede referirse a una primaria o secundaria

In [21]:
cd2 = a2[["ActivityCaptureSer","CourseSer","ActivityInstanceSer"]] \
.merge(a1[["DiagnosisSer","ActivityCaptureSer"]], on="ActivityCaptureSer", how="outer") \
[["CourseSer","DiagnosisSer"]].drop_duplicates().dropna()

cdf = pd.concat([cd, cd2])[["CourseSer","DiagnosisSer"]].drop_duplicates()

In [22]:
# me quedo con aquellas relaciones Course-Diagnosis donde el curso solo tiene asociada una diagnosis
vc = cdf[cdf.DiagnosisSer.isin(diag_pri.DiagnosisSer)].CourseSer.value_counts()
c1 = vc[vc==1].index

vc = cdf[cdf.DiagnosisSer.isin(diag_sec.DiagnosisSer)].CourseSer.value_counts()
c2 = vc[vc==1].index

cd_pri = cdf[(cdf.CourseSer.isin(c1)) & cdf.DiagnosisSer.isin(diag_pri.DiagnosisSer)]
cd_sec = cdf[(cdf.CourseSer.isin(c2)) & cdf.DiagnosisSer.isin(diag_sec.DiagnosisSer)]

In [23]:
print(cd_pri.shape)
print(cd_pri.CourseSer.nunique())
print(cd_pri.DiagnosisSer.nunique())

a,b,c = ut.rel_indices(cd_sec.CourseSer, cur.CourseSer)

(4589, 2)
4589
4305
Longitud del primer set: 663
Longitud del segundo set: 6430

Indices comunes: 663
Indices que solo están en el primero: 0
Indices que solo están en el segundo: 5767
_____________________________________________



# Join

In [50]:
cur_pi_diag = cur\
.merge(pi, on="CourseSer", how="left", suffixes = ("_cur", "_pi")).drop(columns=["PhysicianIntentSer"])\
.merge(cd_pri, on="CourseSer", how="left", suffixes = ("_curpi", "_cd_pri"))\
.merge(diag_pri, on="DiagnosisSer", how="left", suffixes = ("_curpi", "_diag_pri")).drop(columns="DiagnosisSer")\
.merge(cd_sec, on="CourseSer", how="left", suffixes = ("_curpi", "_cd_sec"))\
.merge(diag_sec, on="DiagnosisSer", how="left", suffixes = ("_curpi", "_diag_sec")).drop(columns="DiagnosisSer")

# 71%
cur_pi_diag.shape

(6430, 15)

In [78]:
print(diag.shape)
diag.isna().sum()

(5428, 6)


DiagnosisSer        0
DiagnosisType       0
PatientSer          0
HistologyCode    3769
DiagnosisCode       0
ObjectStatus        0
dtype: int64

# Resumen

In [25]:
ut.resumen_tabla(cur_pi_diag, 'nans')

6430 registros
15 variables


,nans,nunicos,nunicos_r,dtype
PatientSer,0.00,5413,84.18,int64
CourseSer,0.00,6430,100.00,int64
ClinicalStatus,0.00,2,0.03,object
StartDateTime,3.00,6236,96.98,object
TreatmentIntentType,7.84,15,0.23,object
DiagnosisCode,28.63,422,6.56,object
ObjectStatus,28.63,2,0.03,object
SummaryStage,42.75,26,0.40,object
TStage,49.61,80,1.24,object
MStage,51.57,13,0.20,object


# Archivo final

In [26]:
cur_pi_diag.to_csv("data/ARIA_merged&selected/Course_pi_diag.csv", index=False)